# First Lookup on DemoParse Library


In [23]:
from demoparser2 import DemoParser
import pandas as pd
from pprint import pprint
from typing import Dict, Sequence, Optional, List, Tuple
import numpy as np

In [24]:
base_path = "../../demos"
parser = DemoParser(f"{base_path}/faceit/anubisFaceit.dem")

event_names = parser.list_game_events()

# Currently the event "all" gives you all events. Cursed solution for now
event_names

['player_death',
 'round_poststart',
 'cs_pre_restart',
 'round_announce_warmup',
 'cs_win_panel_match',
 'round_announce_match_start',
 'cs_round_start_beep',
 'cs_round_final_beep',
 'player_jump',
 'other_death',
 'player_team',
 'player_footstep',
 'round_prestart',
 'chat_message',
 'round_time_warning',
 'bomb_defused',
 'player_connect',
 'item_pickup',
 'player_connect_full',
 'round_announce_last_round_half',
 'weapon_fire',
 'server_cvar',
 'player_spawn',
 'smokegrenade_detonate',
 'weapon_reload',
 'inferno_startburn',
 'bomb_begindefuse',
 'hltv_chase',
 'hegrenade_detonate',
 'bomb_exploded',
 'player_hurt',
 'bomb_pickup',
 'bomb_planted',
 'smokegrenade_expired',
 'bomb_beginplant',
 'buytime_ended',
 'bomb_dropped',
 'hltv_fixed',
 'decoy_detonate',
 'decoy_started',
 'inferno_expire',
 'player_disconnect',
 'round_announce_final',
 'player_blind',
 'round_officially_ended',
 'begin_new_match',
 'server_message',
 'show_survival_respawn_status',
 'flashbang_detonate',


# Game Score

Halftime or final score


In [25]:
def get_player_average_damage_per_round():
    """
    Function to calculate the Average Damage per Round (ADR) for all players.

    Args:
        parser (DemoParser): An instance of DemoParser initialized with a demo file path.

    Returns:
        pd.DataFrame: A dataframe containing player Steam IDs and their ADR.
    """
    # Define the fields to extract
    wanted_fields = ["damage_total", "kills_total", "deaths_total"]

    # Parse the number of rounds
    round_end_df = parser.parse_event("round_end")
    return len(round_end_df) - 1  # Count the total number of rounds

    # # Parse the maximum tick to analyze final stats
    # max_tick = round_end_df["tick"].max()

    # # Parse the wanted fields for all players at the last tick
    # stats_df = parser.parse_ticks(wanted_fields, ticks=[max_tick])

    # # Ensure no NaN values in the stats
    # stats_df.fillna(0, inplace=True)

In [82]:
def get_final_score(
    players: Sequence[str] = None,
    round_info: str | int = "final",
) -> Tuple[pd.DataFrame, pd.DataFrame] | pd.DataFrame:
    """
    Retrieve the final score details for specified players or all players at the end of a round.

    Parameters:
    -----------
    players : Sequence[str], optional
        A list of player names to filter the DataFrame. If provided, the returned DataFrame 
        will only include data for the specified players. Defaults to None.

    round_info : str | int, optional
        Specifies the round for which to retrieve score data. Can take the following values:
        - "final" (default): Retrieves the final round score.
        - "half_time": Retrieves the score at halftime (end of round 12).
        - An integer: Retrieves the score for the specified round (e.g., 1 for the first round).
        If the integer is greater than the maximum round available, a ValueError is raised.

    Returns:
    --------
    Union[Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame], pd.DataFrame]
        - If `players` is provided: A single filtered DataFrame containing data for the specified players.
        - Otherwise: A tuple containing:
            1. DataFrame for Team 1
            2. DataFrame for Team 2
            3. DataFrame containing data for all players, sorted by team and KD ratio.

    Raises:
    -------
    ValueError
        - If `round_info` is not "final", "half_time", or a valid integer.
        - If `round_info` is an integer greater than the maximum round available.
    """

    
     # Determine the tick based on round_info
    events = parser.parse_event("round_end")
    max_round = len(events) - 1
    if isinstance(round_info, int):
        if round_info > max_round:
            raise ValueError(f"Invalid `round_info`: {round_info}. Maximum round is {max_round}.")
        tick = events.query(f"round == {round_info + 2}")["tick"].max()
    elif round_info == "half_time":
        tick = events.query("round == 14")["tick"].max()
    elif round_info == "final":
        tick = events["tick"].max()
    else:
        raise ValueError("Invalid `round_info` value. Must be 'final', 'half_time', or an integer.")

    wanted_fields = [
        "kills_total",
        "deaths_total",
        "mvps",
        "headshot_kills_total",
        "ace_rounds_total",
        "4k_rounds_total",
        "3k_rounds_total",
        "team_num",
        "damage_total",
        "assists_total",
        "team_score_first_half",
        "team_score_second_half",
    ]

    # Parse the ticks at the max_tick
    df = parser.parse_ticks(wanted_fields, ticks=[tick])    
    df["deaths_total"] = df["deaths_total"].fillna(0)

    # Calculate KD
    df["kd"] = np.where(
        df["deaths_total"] != 0,  
        round(df["kills_total"] / df["deaths_total"], 2),
        round((df["kills_total"] / 1), 2),
    )

    # Calculate HS %
    df["headshot_percentage"] = np.where(
    df["kills_total"] != 0,  
    round(df["headshot_kills_total"] / df["kills_total"] * 100),
    round((df["headshot_kills_total"] / 1) * 100),
    ).astype(int)   

    # Calculate ADR
    first_half_rounds = df["team_score_first_half"].unique().sum()
    second_half_rounds = df["team_score_second_half"].unique().sum()
    actual_rounds = first_half_rounds + second_half_rounds
    
    
    print(actual_rounds)
    if actual_rounds == 0:
        df["adr"] = round(df["damage_total"] / 1).astype(int)
    else:
        df["adr"] = round(df["damage_total"] / actual_rounds, 2)
    df["round"] = actual_rounds
    
    df["diff"] = df["kills_total"] - df["deaths_total"]
    
    df.sort_values("kd", inplace=True, ascending=False)
    
    # If specific players are provided, filter the DataFrame and return
    if players:
        df = df[df["name"].isin(players)]
        return df

    # Get unique team numbers dynamically
    unique_teams = df["team_num"].unique()

    # Assign the team numbers dynamically
    team_num_1, team_num_2 = unique_teams[:2]

    # Create separate DataFrames for each team
    df_team_1: pd.DataFrame = df[df["team_num"] == team_num_1].copy()
    df_team_2: pd.DataFrame = df[df["team_num"] == team_num_2].copy()

    return df_team_1, df_team_2, df.sort_values("team_num")


df = get_final_score(round_info=0)
df[2]

0


,kills_total,deaths_total,assists_total,headshot_kills_total,damage_total,ace_rounds_total,4k_rounds_total,3k_rounds_total,mvps,team_num,team_score_first_half,team_score_second_half,tick,steamid,name,kd,headshot_percentage,adr,round,diff
3,2,0,0,0,0,0,0,0,0,2,0,0,2333,76561198077572982,cebola1_,2.0,0,0,0,2
7,2,0,0,0,0,0,0,0,1,2,0,0,2333,76561198437803219,creepyfps,2.0,0,0,0,2
4,1,1,0,0,0,0,0,0,0,2,0,0,2333,76561198317791204,MIRAcls_,1.0,0,0,0,0
0,0,1,1,0,0,0,0,0,0,2,0,0,2333,76561198803359038,EvUsFDL,0.0,0,0,0,-1
6,0,1,1,0,0,0,0,0,0,2,0,0,2333,76561199075107764,PuliNFPS,0.0,0,0,0,-1
9,2,1,0,0,0,0,0,0,0,3,0,0,2333,76561198129776170,brbzinN,2.0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,3,0,0,2333,76561198840306307,ton001,1.0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,3,0,0,2333,76561197999127404,vap999,0.0,0,0,0,-1
5,0,1,1,0,0,0,0,0,0,3,0,0,2333,76561197987161730,sr_jarbolas,0.0,0,0,0,-1
8,0,1,1,0,0,0,0,0,0,3,0,0,2333,76561198057428801,snowwJL,0.0,0,0,0,-1
